### In this notebook we are going to cover some of the most fundamental concepts of tensors using tensorflow.

More specifically we are going to cover:
* Introduction to tesors
* Getting information from tensors
* Manipulating tesors
* Tensors and Numpy
* Using @tf.function (a wat to speed up the regular Python functions)
* Using GPUs with Tensorflow or TPUs
* Exercises to try

## Introduction to Tensors

In [70]:
# Import Tensorflow
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.3.0


In [6]:
# create tensor with tf.constant()
scaler = tf.constant(7)
scaler

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [7]:
# check the number of dimensions of a tensor (ndim stand for number of dimension)

scaler.ndim


0

In [8]:
# Create a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10])>

In [10]:
# check the domension of the vector
vector.ndim

1

In [11]:
# create a matrix - more than one dimension
matrix = tf.constant([[10,7],
                     [7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]])>

In [12]:
matrix.ndim

2

In [16]:
# Create another matrix
matrix2 = tf.constant([[10.,5.],
                      [3.,4.],
                      [5.,6.]], dtype = tf.float16) # specify the datatype with dtype
matrix2

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  5.],
       [ 3.,  4.],
       [ 5.,  6.]], dtype=float16)>

In [17]:
matrix2.ndim

2

In [18]:
# Lets create a tensor
tensor = tf.constant([[[1, 2, 3],
                      [4, 5, 6]],
                      [[7, 8, 9],
                       [10,11,12]],
                      [[13,14,15],
                      [16,17,18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]])>

In [20]:
tensor.ndim

3

Scaler is a single number

* Vector is a number with direction (e.g wind speed with direction)
* Matrix  is a 2-D array of numbers
* Tensor is a n-dimension array of numbers(when n can be any number; 0-dimensional  tensor is a scaler, 
  1-dimensional is a vector)

In [23]:
# Creating Tensors using tf.Variable
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor,unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7])>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7])>)

In [24]:
# Lets try to change the element in the tensor
changeable_tensor[0] = 7

TypeError: 'ResourceVariable' object does not support item assignment

In [26]:
#How about we using v.assign
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7])>

# Creating Random Tensors 
Rarely in practise you need to decide whether to use tf.contant or tf.Variable to create tensor, as Tensorflow does this for you.
When in doubt always go for constant tensors and change it later it needed.

In [80]:
# Create two random but same tensors
random_1 = tf.random.Generator.from_seed(42) # Set seed for reproduciability
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.uniform(shape=(3,2))

# Are they equal 
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0.7493447 , 0.73561966],
        [0.45230794, 0.49039817],
        [0.1889317 , 0.52027524]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[False, False],
        [False, False],
        [False, False]])>)

### Shuffle the order of elements in a tensor

In [39]:
# Shuffle a tensor(valuable for when you want to shuffle the data during training)
not_shuffled = tf.constant([[4,5],
                           [6,7],
                           [8,9]])
# shuffle your tensor



In [41]:
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed=42) # shuffled along the first dimension

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[4, 5],
       [6, 7],
       [8, 9]])>

In [42]:
random_3 = tf.constant([[8,9],
                       [7,5],
                       [4,7]])

In [68]:
tf.random.set_seed(42) #GLobal level random seed
tf.random.shuffle(random_3, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[8, 9],
       [7, 5],
       [4, 7]])>

In [72]:
#Other ways to make sensors
tf.ones([10,3])

<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [74]:
# Create tensors of all zeroes
tf.zeros(shape=[10,4])

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn Numpy arrays into tensors
The main difference between numpy arrays and tensors is that tensors can be run on a GPU(much faster for computing)

In [77]:
# You can also convert numpy into tensors
numpy_a = np.arange(1,25, dtype=np.int32)
numpy_a
# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector)  # non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [82]:
A = tf.constant(numpy_a,shape= (2,3,4))
B = tf.constant(numpy_a, shape = (4,6))

A,B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]])>,
 <tf.Tensor: shape=(4, 6), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6],
        [ 7,  8,  9, 10, 11, 12],
        [13, 14, 15, 16, 17, 18],
        [19, 20, 21, 22, 23, 24]])>)

In [84]:
# Getting information from tensors
array1 = np.array([1,2,3,4,5,6,7,7,8])

In [89]:
C = tf.constant(array1,shape=(1,3,3))
C

<tf.Tensor: shape=(1, 3, 3), dtype=int32, numpy=
array([[[1, 2, 3],
        [4, 5, 6],
        [7, 7, 8]]])>

In [91]:
C.ndim,A.ndim,B.ndim

(3, 3, 2)

## Getting information from Tensors
-- Shape
-- Rank
-- Axis or Dimension
-- Size

In [97]:
rank_4_tensor = tf.ones(shape=[2,3,4,5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]],


       [[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]]], dtype=float32)>

In [98]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]], dtype=float32)>

In [101]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [106]:
# Get various attributes of our tensors
print("Datatype of every element", rank_4_tensor.dtype)
print("Number of dimensions", tf.size(rank_4_tensor))
print("Shape of the tensor", rank_4_tensor.shape)
print("Element along the o axis", rank_4_tensor[0])
print("Element along the last axis", rank_4_tensor[-1])
print("Total number of elements in the tensors are", tf.size(rank_4_tensor))
print("Total number of elements in the tensors are", tf.size(rank_4_tensor).numpy())

Datatype of every element <dtype: 'float32'>
Number of dimensions tf.Tensor(120, shape=(), dtype=int32)
Shape of the tensor (2, 3, 4, 5)
Element along the o axis tf.Tensor(
[[[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]], shape=(3, 4, 5), dtype=float32)
Element along the last axis tf.Tensor(
[[[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]], shape=(3, 4, 5), dtype=float32)
Total number of elements in the tensors are tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in the tensors are 120


### Indexing Tensors
Tensors can be indexed just like Python list.

In [110]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2,:2,:2,:2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]]],


       [[[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]]]], dtype=float32)>

In [112]:
rank_2_tensor = tf.constant([[1,3,5],
                            [2,4,7]])
rank_2_tensor

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 3, 5],
       [2, 4, 7]])>

In [115]:
tf.rank(rank_2_tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [118]:
rank_2_tensor[:,-1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([5, 7])>

In [122]:
rank_3_tensor = rank_2_tensor[...,tf.newaxis]

In [120]:
rank_3_tensor

<tf.Tensor: shape=(2, 3, 1), dtype=int32, numpy=
array([[[1],
        [3],
        [5]],

       [[2],
        [4],
        [7]]])>

In [121]:
rank_3_tensor.shape

TensorShape([2, 3, 1])

In [124]:
#Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor,axis=-1)  # "-1" means expand the final axis

<tf.Tensor: shape=(2, 3, 1), dtype=int32, numpy=
array([[[1],
        [3],
        [5]],

       [[2],
        [4],
        [7]]])>

In [125]:
tf.expand_dims(rank_2_tensor, axis=0)

<tf.Tensor: shape=(1, 2, 3), dtype=int32, numpy=
array([[[1, 3, 5],
        [2, 4, 7]]])>

### Manipulating tensor, tensor operations

In [127]:
rank_2_tensor

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 3, 5],
       [2, 4, 7]])>

**Basic Operations**
+,-,*,/

In [128]:
tensor = tf.constant([[4,5],[8,9],[7,9]])
tensor + 10

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[14, 15],
       [18, 19],
       [17, 19]])>

In [130]:
tensor = tensor + 1
tensor

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 6,  7],
       [10, 11],
       [ 9, 11]])>

In [133]:
# Multiplication
tensor*19,tensor-11, tensor/2

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[114, 133],
        [190, 209],
        [171, 209]])>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[-5, -4],
        [-1,  0],
        [-2,  0]])>,
 <tf.Tensor: shape=(3, 2), dtype=float64, numpy=
 array([[3. , 3.5],
        [5. , 5.5],
        [4.5, 5.5]])>)

In [134]:
tf.multiply(tensor,10)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 60,  70],
       [100, 110],
       [ 90, 110]])>

In [135]:
tensor

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 6,  7],
       [10, 11],
       [ 9, 11]])>

### Matrix multiplication
In machine learning matrix multiplication is basic and important part of the process

In [138]:
# Dot product of two matrix
print(tensor)
tf.matmul(tensor,tf.transpose(tensor))

tf.Tensor(
[[ 6  7]
 [10 11]
 [ 9 11]], shape=(3, 2), dtype=int32)


<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 85, 137, 131],
       [137, 221, 211],
       [131, 211, 202]])>

In [140]:
##In python we can bascially use '@' for matrix multiplication
tensor @ tf.transpose(tensor)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 85, 137, 131],
       [137, 221, 211],
       [131, 211, 202]])>

** Dot product **
We can perform this multiplication using:
tf.matmul
tf.tensordot

In [142]:
tf.tensordot(tensor, tf.transpose(tensor), axes=1)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 85, 137, 131],
       [137, 221, 211],
       [131, 211, 202]])>

In [144]:
tf.tensordot(tensor, tf.reshape(tensor, shape=(2,3)), axes=1)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[113, 105, 137],
       [181, 169, 221],
       [175, 162, 211]])>

In [145]:
M =  tf.constant([1.5,5.6])
M.dtype

tf.float32

In [146]:
N = tf.cast(M, dtype=tf.float16)
N, N.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.5, 5.6], dtype=float16)>,
 tf.float16)

### Aggregating Tensors

In [148]:
# Get the absolute values

In [149]:
N = tf.constant([-1,-9])
N

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([-1, -9])>

In [150]:
tf.abs(N)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 9])>

In [156]:
tensor

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 6,  7],
       [10, 11],
       [ 9, 11]])>

In [154]:
# Mean of tensor
tf.reduce_mean(tensor), tf.reduce_max(tensor), tf.reduce_min(tensor), tf.reduce_sum(tensor)

(<tf.Tensor: shape=(), dtype=int32, numpy=9>,
 <tf.Tensor: shape=(), dtype=int32, numpy=11>,
 <tf.Tensor: shape=(), dtype=int32, numpy=6>,
 <tf.Tensor: shape=(), dtype=int32, numpy=54>)

In [161]:
tf.math.reduce_std(tf.cast(tensor,dtype=tf.float16))

<tf.Tensor: shape=(), dtype=float16, numpy=1.915>

In [162]:
tf.math.reduce_variance(tf.cast(tensor,dtype=tf.float16))

<tf.Tensor: shape=(), dtype=float16, numpy=3.666>

In [164]:
## find the positional maximum and minimum
tf.argmax(tensor), tf.argmin(tensor)

(<tf.Tensor: shape=(2,), dtype=int64, numpy=array([1, 1], dtype=int64)>,
 <tf.Tensor: shape=(2,), dtype=int64, numpy=array([0, 0], dtype=int64)>)

In [168]:
# Squeezing of tensors
tf.random.set_seed(42)
M = tf.random.uniform(shape=(1,1,1,1,1,1,50),seed=42)
M

<tf.Tensor: shape=(1, 1, 1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[[[0.4163028 , 0.26858163, 0.47968316, 0.36457133,
             0.95471144, 0.9418646 , 0.61483395, 0.35842144,
             0.5936024 , 0.21551096, 0.07745171, 0.57921314,
             0.29180396, 0.26718032, 0.37012458, 0.7161033 ,
             0.45877767, 0.11764562, 0.21073711, 0.5441973 ,
             0.9898069 , 0.38395858, 0.04683566, 0.8718462 ,
             0.25881708, 0.873135  , 0.64698434, 0.41981232,
             0.24148273, 0.09550059, 0.9820819 , 0.1570208 ,
             0.2997682 , 0.36795306, 0.9453716 , 0.11056781,
             0.52287626, 0.8305441 , 0.0020721 , 0.9594034 ,
             0.85630023, 0.3944497 , 0.22028875, 0.67066073,
             0.01875746, 0.48057055, 0.5953454 , 0.6847329 ,
             0.18988943, 0.12489867]]]]]]], dtype=float32)>

In [169]:
tf.squeeze(M)

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.4163028 , 0.26858163, 0.47968316, 0.36457133, 0.95471144,
       0.9418646 , 0.61483395, 0.35842144, 0.5936024 , 0.21551096,
       0.07745171, 0.57921314, 0.29180396, 0.26718032, 0.37012458,
       0.7161033 , 0.45877767, 0.11764562, 0.21073711, 0.5441973 ,
       0.9898069 , 0.38395858, 0.04683566, 0.8718462 , 0.25881708,
       0.873135  , 0.64698434, 0.41981232, 0.24148273, 0.09550059,
       0.9820819 , 0.1570208 , 0.2997682 , 0.36795306, 0.9453716 ,
       0.11056781, 0.52287626, 0.8305441 , 0.0020721 , 0.9594034 ,
       0.85630023, 0.3944497 , 0.22028875, 0.67066073, 0.01875746,
       0.48057055, 0.5953454 , 0.6847329 , 0.18988943, 0.12489867],
      dtype=float32)>

In [171]:
tensor

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 6,  7],
       [10, 11],
       [ 9, 11]])>

In [179]:
## One hot encoding of tensors
one_hot = [1,2,3,5]
tf.one_hot(one_hot,depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.]], dtype=float32)>


### Finding Access to GPU

In [184]:
tf.config.list_physical_devices("CPU")

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

** If you have access to CUDA enabled GPU, tensorflow will automatically use it in computation **
<br>
** CUDA is a parallel computing platform and application programming interface that allows software to use certain types of graphics processing unit for general purpose processing. **